In [ ]:
%matplotlib inline

proj_path = "/Users/andrew/Desktop/projects/hidden_singles_public/"

In [ ]:
import sys
sys.path.append(proj_path + 'python')

import json
import pandas as pd
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from PIL import Image, ImageDraw, ImageFont
from tqdm.auto import tqdm

from hiddensingles.misc import utils
from hiddensingles.sudoku.grid import Grid, GridString,Coordinate

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 300)

In [ ]:
c_bg_green = (1, 255, 112)
c_bg_blue = (127, 219, 255)
c_bg_purple = (218, 112, 214)
c_bg_red = (255, 69, 0)
c_bg_orange = (255, 165, 0)
c_digit_blue = (0, 0, 255)
line_house_width = 4
line_cell_width = 1

In [ ]:
# Same code as from Data Wrangler

subject_ids = {}
def load_raw_data(dirname, subject_ids):
    """
    Parses raw data into a dictionary
    return:
        a dictionary of (subject_id, data)
        list of subject_ids whose data could not be parsed
    """
    raw_data = {}
    failures = []
    for filename in tqdm(sorted(glob.glob(dirname + '/*'))):
        try:
            with open(filename) as f:
                data = json.load(f)
            worker_id = os.path.basename(filename)
            if completed_hit(data):
                if worker_id not in subject_ids:
                    subject_ids[worker_id] = len(subject_ids)
                raw_data[subject_ids[worker_id]] = data
                data['worker_id'] = worker_id
        except:
            failures.append(filename)
    if failures:
        worker_ids = ['"{}"'.format(os.path.basename(f)) for f in failures]
        worker_ids = ', '.join(worker_ids)
        print("Failed to open {} files: {}".format(len(failures), worker_ids))
    return raw_data

def parse_timestring(s):
    return datetime.strptime(s, '%Y%m%d_%H%M%S%f_%Z')

def completed_hit(data):
    return bool([d for d in data['data'] if d['screen'] == 'MathBackgroundSurvey'])

In [ ]:
def get_coordinates(hidden_single_raw):
    coords = {}
    for k, v in hidden_single_raw['coordinates'].items():
        if type(v) == list:
            coords[k] = [Coordinate(c['x'], c['y']) for c in v]
        else:
            coords[k] = Coordinate(v['x'], v['y'])
    return coords

In [ ]:
def img_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def img_concat_v(im1, im2):
    dst = Image.new('RGB', (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

def img_add_title(img, title, bg_color=(255, 255, 255)):
    img_title = Image.new('RGB', (img.width, int(.12*img.height)), color = bg_color)
    draw = ImageDraw.Draw(img_title)
    fnt = ImageFont.truetype('/Library/Fonts/Arial.ttf', int(.65*img_title.height))
    text_w, text_h = draw.textsize(title, font=fnt) 
    x = int((img_title.width - text_w))/2
    y = int((img_title.height - text_h))/2
    draw.text((x, y), title, font=fnt, fill='black')
    return img_concat_v(img_title, img)

def get_house_highlights(house_type, goal):
    highlights = {Coordinate(i, goal.y) if house_type == 'column' else Coordinate(goal.x, i): c_bg_blue for i in range(9)}
    return highlights

def render_sudoku(gridstring, highlights={}, size=400):
    numbers = GridString.load(gridstring).get_hints()
    
    # Rendering
    img = Image.new('RGB', (size, size), color = (255, 255, 255))

    img_max = size-2
    house_width = int(img_max/3)
    cell_width = int(house_width/3)

    fnt = ImageFont.truetype('/Library/Fonts/Arial.ttf', int(.1*size))
    draw = ImageDraw.Draw(img)

    # Add highlighted cells
    for coord, color in highlights.items():
        if color is not None:
            x, y = coord.x, coord.y
            draw.rectangle((y*cell_width,
                            x*cell_width,
                            (y+1)*cell_width,
                            (x+1)*cell_width), fill=color)

    # Create exterior borders
    draw.line((0, 0) + (0, img_max), fill=0, width=line_house_width)
    draw.line((0, 0) + (img_max, 0), fill=0, width=line_house_width)
    draw.line((img_max, 0) + (img_max, img_max), fill=0, width=line_house_width)
    draw.line((0, img_max) + (img_max, img_max), fill=0, width=line_house_width)

    # Create house borders
    draw.line((0, house_width) + (img_max, house_width), fill=0, width=line_house_width)
    draw.line((0, 2*house_width) + (img_max, 2*house_width), fill=0, width=line_house_width)
    draw.line((house_width, 0) + (house_width, img_max), fill=0, width=line_house_width)
    draw.line((2*house_width, 0) + (2*house_width, img_max), fill=0, width=line_house_width)

    # Create cell borders
    for i in range(9):
        draw.line((0, i*cell_width) + (img_max, i*cell_width), fill=0, width=line_cell_width)
        draw.line((i*cell_width, 0) + (i*cell_width, img_max), fill=0, width=line_cell_width)

    # Add numerals
    for coord, num in numbers.items():
        x, y = coord.x, coord.y
        draw.text(((y+.27)*cell_width,
                   (x+.025)*cell_width), str(num),
                   font=fnt, fill=c_digit_blue)
        
    return img

def render_hidden_single(hidden_single_raw, size=400):
    house_type = hidden_single_raw['houseType']
    gridstring = hidden_single_raw['gridstrings']['puzzle']
    coordinates = get_coordinates(hidden_single_raw)
    goal = coordinates['goal']
    highlights = get_house_highlights(house_type, goal)
    
    # Format highlights as dictionary: {(x, y): (R, G, B)}
    column = house_type == "column"
    highlights[goal] = c_bg_green
    
    return render_sudoku(gridstring, highlights, size)
    


# Load Data

In [ ]:
# Load raw_data and map them using sid_hash
raw_data = load_raw_data(proj_path + 'data/raw', subject_ids)
sid_hashes = utils.short_hash([d['worker_id'] for d in raw_data.values()], 6)
raw_data = {sid_hashes[k]: v for k, v in raw_data.items()}

subject_data = pd.read_csv(proj_path + 'data/processed/subject_data.tsv', sep='\t')
experiment = raw_data[sid_hashes[1]]['experimentDetails'] # all figures generated using this person's data

# Exercises

In [ ]:
tutorial = experiment['tutorial']
tut_coords = get_coordinates(tutorial)
goal_coords = tut_coords['goal']

contradiction = experiment['contradiction']
img = render_sudoku(contradiction)
img.save(proj_path + 'figures/sample_puzzles/contradiction.png')

In [ ]:
fullhouse = experiment['fullhouse']
highlights = {goal_coords: c_bg_green}
img = render_sudoku(fullhouse, highlights)
img.save(proj_path + 'figures/sample_puzzles/fullhouse.png')

# Tutorial

In [ ]:
# The actual tutorial puzzle

gridstring = tutorial['gridstrings']['puzzle']
highlights = get_house_highlights(tutorial['houseType'], goal_coords)
highlights[goal_coords] = c_bg_green
img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_0.png')

In [ ]:
# Tutorial screen 1 & 5

grid = GridString.load(tutorial['gridstrings']['puzzle']).to_grid()
dist1 = tut_coords['distractorSingle']
dist2 = tut_coords['distractorDouble']
dist3 = tut_coords['distractorBox']
for coord in [dist1, dist2, dist3]:
    grid.remove(coord)
gridstring = str(grid.to_gridstring())

highlights = get_house_highlights(tutorial['houseType'], goal_coords)
highlights[goal_coords] = c_bg_green

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_1.png')

In [ ]:
# Tutorial screen 2
highlights[tut_coords['emptyDouble']] = c_bg_purple
for coord in tut_coords['occupied']:
    highlights[coord] = c_bg_red

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_2.png')

In [ ]:
# Tutorial screen 3
highlights[tut_coords['emptyDouble']] = c_bg_red
for coord in tut_coords['emptyBox']:
    highlights[coord] = c_bg_purple

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_3.png')

In [ ]:
# Tutorial screen 4
highlights[tut_coords['emptySingle']] = c_bg_purple
for coord in tut_coords['emptyBox']:
    highlights[coord] = c_bg_red

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_4.png')

In [ ]:
# Tutorial screen 6
gridstring = tutorial['gridstrings']['puzzle']
highlights = get_house_highlights(tutorial['houseType'], goal_coords)
highlights[goal_coords] = c_bg_green
for coord in tut_coords['occupied']:
    highlights[coord] = c_bg_red
highlights[tut_coords['distractorDouble']] = c_bg_purple

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_6.png')

In [ ]:
# Tutorial screen 7
highlights[tut_coords['distractorDouble']] = None
highlights[tut_coords['emptyDouble']] = c_bg_red
highlights[tut_coords['distractorBox']] = c_bg_purple

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_7.png')

In [ ]:
# Tutorial screen 8
highlights[tut_coords['distractorBox']] = None
for coord in tut_coords['emptyBox']:
    highlights[coord] = c_bg_red

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_8.png')

In [ ]:
# Tutorial screen 9a
grid = GridString.load(tutorial['gridstrings']['puzzle']).to_grid()
highlights = get_house_highlights(tutorial['houseType'], goal_coords)
highlights[goal_coords] = c_bg_green
highlights[tut_coords['targetSingle']] = c_bg_orange
highlights[tut_coords['targetDouble']] = c_bg_orange
highlights[tut_coords['emptyDouble']] = c_bg_orange
highlights[tut_coords['emptySingle']] = c_bg_orange
highlights[tut_coords['targetBox']] = c_bg_purple
for coord in tut_coords['emptyBox']:
    highlights[coord] = c_bg_purple
for coord in tut_coords['occupied']:
    highlights[coord] = c_bg_red

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_9a.png')

In [ ]:
# Tutorial screen 9b
grid = GridString.load(tutorial['gridstrings']['puzzle']).to_grid()
highlights = get_house_highlights(tutorial['houseType'], goal_coords)
highlights[goal_coords] = c_bg_green
highlights[tut_coords['distractorSingle']] = c_bg_orange
highlights[tut_coords['distractorDouble']] = c_bg_orange
highlights[tut_coords['emptyDouble']] = c_bg_orange
highlights[tut_coords['distractorBox']] = c_bg_purple
for coord in tut_coords['emptyBox']:
    highlights[coord] = c_bg_purple
for coord in tut_coords['occupied']:
    highlights[coord] = c_bg_red

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_9b.png')

# Practice Phase

In [ ]:
# Sample puzzles

for i in range(4):
    img = render_hidden_single(experiment['phase1'][i])
    img.save(proj_path + 'figures/sample_puzzles/p1_puzzle_{}.png'.format(i+1))

In [ ]:
# In-House feedback

hidden_single = experiment['phase1'][0]
gridstring = hidden_single['gridstrings']['puzzle']
coords = get_coordinates(hidden_single)
goal = coords['goal']
highlights = get_house_highlights(hidden_single['houseType'], goal)
highlights[goal] = c_bg_green
highlights[coords['occupied'][0]] = c_bg_red
img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/p1_feedback_inhouse.png')

In [ ]:
# Absent Feedback

highlights = get_house_highlights(hidden_single['houseType'], goal)
highlights[goal] = c_bg_green
for coord in coords['occupied']:
    highlights[coord] = c_bg_red
img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/p1_feedback_absent.png')

In [ ]:
# Distractor Feedback

highlights = get_house_highlights(hidden_single['houseType'], goal)
highlights = {k: c_bg_red for k in highlights}
highlights[goal] = c_bg_green
highlights[coords['emptySingle']] = c_bg_blue
img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/p1_feedback_distractor.png')

In [ ]:
# Target feedback

highlights = get_house_highlights(hidden_single['houseType'], goal)
highlights[goal] = c_bg_green
for coord in coords['occupied']:
    highlights[coord] = c_bg_red
for coord in coords['emptyBox']:
    highlights[coord] = c_bg_purple
highlights[coords['emptySingle']] = c_bg_orange
highlights[coords['emptyDouble']] = c_bg_orange

highlights[coords['targetBox']] = c_bg_purple
highlights[coords['targetSingle']] = c_bg_orange
highlights[coords['targetDouble']] = c_bg_orange

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/p1_feedback_target.png')

# Test Phase

In [ ]:
# Test Phase

for i in range(16):
    hidden_single = experiment['phase2'][i]
    condition = hidden_single['condition']
    
    title = []
    if condition['houseType']:
        title.append('ht')
    if condition['houseIndex']:
        title.append('hi')
    if condition['cellIndex']:
        title.append('ci')
    if condition['digitSet']:
        title.append('ds')
    title = ''.join(title) if title else 'control'
    
    img = render_hidden_single(hidden_single)

    img.save(proj_path + 'figures/sample_puzzles/p2_puzzle_{}.png'.format(title))

# Questionnaire

In [ ]:
for record in tqdm(subject_data[~subject_data.excluded].to_records()):
    subject_id = record.subject_id
    sid_hash = record.sid_hash
    hidden_single = raw_data[sid_hash]['experimentDetails']['questionnaire']
    img = render_hidden_single(hidden_single)
    title = 'Questionnaire (Subject {})'.format(subject_id)
    img = img_add_title(img, title)
    img.save(proj_path + 'figures/questionnaire_puzzles/{}.png'.format(subject_id))